# Optimize XG Boost - parameters / hyper parameters

In [1]:
import pandas as pd
import numpy as np
import researchpy as rp
from scipy import stats

import matplotlib.pyplot as plt
import pydotplus
from matplotlib import rcParams
import seaborn as sb


In [2]:
# # Find no. of factors in y
from collections import Counter
# #label encoding
from sklearn.preprocessing import LabelEncoder, scale, MinMaxScaler, StandardScaler,OneHotEncoder,LabelBinarizer
#from sklearn.decomposition import PCA

In [89]:
# Import All classifcation Algorithms
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [4]:
from sklearn.model_selection import train_test_split, GridSearchCV,cross_val_score,validation_curve,KFold

In [5]:
# Metrics selection
from sklearn.metrics import  accuracy_score, recall_score, confusion_matrix,precision_recall_fscore_support,classification_report,roc_curve,auc

In [6]:
# Open files Both files are opened as ny drop or changes in feature will be done is both normal and one hot file
df1 = pd.read_csv('3_ITSM_cleansed_Encoded_Engineered_All.csv')
df1.columns

Index(['c_CI_Cat', 'c_CI_Name', 'c_CI_Subcat', 'c_Closure_Code', 'c_WBS',
       'q_No_of_Reassignments', 'q_No_of_Related_Incidents',
       'q_No_of_Related_Interactions', 't_Close_Time', 't_Open_Time',
       'y_Priority', 't_ReopenFlag', 't_TicketWIPDurationDays'],
      dtype='object')

In [8]:
CatVarList = ['c_CI_Cat', 'c_CI_Name', 'c_CI_Subcat', 'c_Closure_Code', 'c_WBS','t_ReopenFlag']
ConVarList = ['q_No_of_Reassignments', 'q_No_of_Related_Incidents','q_No_of_Related_Interactions','t_TicketWIPDurationDays']
DatVarList = ['t_Close_Time', 't_Open_Time']
OutValList = ['y_Priority']

X_All_1 = ['c_CI_Cat', 'c_CI_Name', 'c_CI_Subcat', 'c_Closure_Code', 'c_WBS','t_ReopenFlag',
           'q_No_of_Reassignments', 'q_No_of_Related_Incidents','q_No_of_Related_Interactions','t_TicketWIPDurationDays']           

## Combining P1+P2 and P3,P4,P5 as 1&0 respectively

In [28]:
print(df1[((df1.y_Priority==0) | (df1.y_Priority==1))].y_Priority.count())
print(df1[((df1.y_Priority==2) | (df1.y_Priority==3) | (df1.y_Priority==4))].y_Priority.count())
print(Counter(df1.y_Priority))

df1.loc[((df1.y_Priority==0) | (df1.y_Priority==1)),'y_Priority']= 1
df1.loc[((df1.y_Priority==2) | (df1.y_Priority==3) | (df1.y_Priority==4)),'y_Priority'] = 0

print(df1[((df1.y_Priority==0) | (df1.y_Priority==1))].y_Priority.count())
print(df1[((df1.y_Priority==2) | (df1.y_Priority==3) | (df1.y_Priority==4))].y_Priority.count())
print(Counter(df1.y_Priority))

434
35412
Counter({3: 22023, 4: 8321, 2: 5068, 1: 431, 0: 3})
35846
0
Counter({0: 35412, 1: 434})


## Assign Variables 

In [54]:
# Assign varibale and X and y
X1 = df1[X_All_1]
y1 = df1[OutValList]

print(X1.shape)
print(y1.shape)

(35846, 10)
(35846, 1)


In [55]:
# Split 80:20
split_seed = 10
X1_train, X1_test, y1_train, y1_test = train_test_split(X1,y1,random_state=split_seed, test_size=.2)
print(X1_train.shape)
print(y1_train.shape)
print(X1_test.shape)
print(y1_test.shape)

(28676, 10)
(28676, 1)
(7170, 10)
(7170, 1)


In [56]:
y1_train.head()

,y_Priority
23572,0
163,0
30420,0
21679,0
23989,0


## Balancing the Imbalaced sets

In [57]:
from imblearn.over_sampling import SMOTE
smote = SMOTE()

In [64]:
X1_train_smote, y1_train_smote = smote.fit_sample(X1_train,y1_train)
print(Counter(y1_train.y_Priority))
print(Counter(y1_train_smote))
X1_train.columns


E:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Counter({0: 28325, 1: 351})
Counter({0: 28325, 1: 28325})


Index(['c_CI_Cat', 'c_CI_Name', 'c_CI_Subcat', 'c_Closure_Code', 'c_WBS',
       't_ReopenFlag', 'q_No_of_Reassignments', 'q_No_of_Related_Incidents',
       'q_No_of_Related_Interactions', 't_TicketWIPDurationDays'],
      dtype='object')

In [66]:
#Convert Smote into Dataframes
X1_train_smote = pd.DataFrame(X1_train_smote,columns=['c_CI_Cat', 'c_CI_Name', 'c_CI_Subcat', 'c_Closure_Code', 'c_WBS',
                                              't_ReopenFlag', 'q_No_of_Reassignments', 'q_No_of_Related_Incidents',
                                              'q_No_of_Related_Interactions', 't_TicketWIPDurationDays'])
y1_train_smote = pd.DataFrame(y1_train_smote,columns=['y_Priority'])
X1_train_smote.shape
y1_train_smote.shape

X1_train_smote.columns
y1_train_smote.columns

Index(['y_Priority'], dtype='object')

## Scaling done based on EDA analysis. Fit on the Train set, but tranform both transform both training and test set

In [81]:
# Scale the data
scaler = StandardScaler()
# Fit on train data
scaler.fit(X1_train_smote[ConVarList])
# Transform both train and test

X1_train_smote[ConVarList] = scaler.transform(X1_train_smote[ConVarList])
X1_train[ConVarList] = scaler.transform(X1_train[ConVarList])
X1_test[ConVarList] = scaler.transform(X1_test[ConVarList])

E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  
E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
E:\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Tr

In [84]:
print(X1_train_smote.shape)
print(X1_train.shape)
print(X1_test.shape)

(56650, 10)
(28676, 10)
(7170, 10)


## Grid Search to optimize the hyper parameters

In [69]:
# Parameters used for Grid search to get best parameters. Use this again only to get best parameters..its take a lot of time.
#parameters = {"learning_rate": [.05,.1, .15],
#              "max_depth": [3,5,8,10,15,20],
#              "min_child_weight" : [ 3, 5, 7 ],
#              "gamma"            : [ 0.0, 0.1],
#              "colsample_bytree" : [ 0.1,0.7,0.8], 
#              "scoring"         : ["recall"] 
#             }
parameters = {"learning_rate": [.1],
              "max_depth": [8],
              "min_child_weight" : [5],
              "gamma"            : [0.1],
              "colsample_bytree" : [.8], 
              "scoring"         : ["recall"] 
             }
model = XGBClassifier(booster='gbtree',random_state=380)
grid_model_XGB = GridSearchCV(model, parameters)
grid_model_XGB.fit(X1_train_smote, y1_train_smote)

E:\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
E:\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
E:\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
E:\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic',
       random_state=380, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'learning_rate': [0.1], 'max_depth': [8], 'min_child_weight': [5], 'gamma': [0.1], 'colsample_bytree': [0.8], 'scoring': ['recall']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [70]:
grid_model_XGB.best_params_

{'colsample_bytree': 0.8,
 'gamma': 0.1,
 'learning_rate': 0.1,
 'max_depth': 8,
 'min_child_weight': 5,
 'scoring': 'recall'}

## XGB with Smotted Data

In [113]:
grid_model_XGB = XGBClassifier(booster='gbtree',random_state=25,scoring='recall')
grid_model_XGB.fit(X1_train_smote, y1_train_smote)
y1_predict = pd.DataFrame(grid_model_XGB.predict(X1_test),columns=['y_Priority'])
print(accuracy_score(y1_test,y1_predict).round(4)*100)
print(recall_score(y1_test,y1_predict).round(4)*100)
print(classification_report(y1_test,y1_predict))
print(confusion_matrix(y1_test,y1_predict))

E:\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
E:\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


97.64
63.85999999999999
              precision    recall  f1-score   support

           0       1.00      0.98      0.99      7087
           1       0.28      0.64      0.39        83

   micro avg       0.98      0.98      0.98      7170
   macro avg       0.64      0.81      0.69      7170
weighted avg       0.99      0.98      0.98      7170

[[6948  139]
 [  30   53]]


## XGB without Smotted Data

In [114]:
grid_model_XGB = XGBClassifier(booster='gbtree',random_state=25,scoring='recall')
grid_model_XGB.fit(X1_train, y1_train)
y1_predict = pd.DataFrame(grid_model_XGB.predict(X1_test),columns=['y_Priority'])
print(accuracy_score(y1_test,y1_predict).round(4)*100)
print(recall_score(y1_test,y1_predict).round(4)*100)
print(classification_report(y1_test,y1_predict))
print(confusion_matrix(y1_test,y1_predict))

E:\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
E:\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


98.8
3.61
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      7087
           1       0.33      0.04      0.07        83

   micro avg       0.99      0.99      0.99      7170
   macro avg       0.66      0.52      0.53      7170
weighted avg       0.98      0.99      0.98      7170

[[7081    6]
 [  80    3]]


## Gradient Boost without smotting 

In [115]:
grid_model_XGB = GradientBoostingClassifier(learning_rate=.1,random_state=25)
grid_model_XGB.fit(X1_train, y1_train)
y1_predict = pd.DataFrame(grid_model_XGB.predict(X1_test),columns=['y_Priority'])
print(accuracy_score(y1_test,y1_predict).round(4)*100)
print(recall_score(y1_test,y1_predict).round(4)*100)
print(classification_report(y1_test,y1_predict))
print(confusion_matrix(y1_test,y1_predict))

E:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


98.22999999999999
18.07
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      7087
           1       0.20      0.18      0.19        83

   micro avg       0.98      0.98      0.98      7170
   macro avg       0.60      0.59      0.59      7170
weighted avg       0.98      0.98      0.98      7170

[[7028   59]
 [  68   15]]


## Gradient Boost with smotting 

In [116]:
grid_model_XGB = GradientBoostingClassifier(learning_rate=.2,random_state=25)
grid_model_XGB.fit(X1_train_smote, y1_train_smote)
y1_predict = pd.DataFrame(grid_model_XGB.predict(X1_test),columns=['y_Priority'])
print(accuracy_score(y1_test,y1_predict).round(4)*100)
print(recall_score(y1_test,y1_predict).round(4)*100)
print(classification_report(y1_test,y1_predict))
print(confusion_matrix(y1_test,y1_predict))

E:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


98.24000000000001
62.64999999999999
              precision    recall  f1-score   support

           0       1.00      0.99      0.99      7087
           1       0.35      0.63      0.45        83

   micro avg       0.98      0.98      0.98      7170
   macro avg       0.67      0.81      0.72      7170
weighted avg       0.99      0.98      0.98      7170

[[6992   95]
 [  31   52]]


In [117]:
print(y1_test.shape)
print(y1_predict.shape)

(7170, 1)
(7170, 1)


In [59]:
store

[[10, 76.0], [20, 76.0], [40, 76.0], [70, 77.60000000000001]]

## Final Model - XGB with Smotting

In [152]:
grid_model_XGB = XGBClassifier(booster='gbtree',random_state=25,scoring='recall')
grid_model_XGB.fit(X1_train_smote, y1_train_smote)
y1_predict = pd.DataFrame(grid_model_XGB.predict(X1_test),columns=['y_Priority'])
print(accuracy_score(y1_test,y1_predict).round(4)*100)
print(recall_score(y1_test,y1_predict).round(4)*100)
print(classification_report(y1_test,y1_predict))
print(confusion_matrix(y1_test,y1_predict))

E:\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
E:\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


97.64
63.85999999999999
              precision    recall  f1-score   support

           0       1.00      0.98      0.99      7087
           1       0.28      0.64      0.39        83

   micro avg       0.98      0.98      0.98      7170
   macro avg       0.64      0.81      0.69      7170
weighted avg       0.99      0.98      0.98      7170

[[6948  139]
 [  30   53]]


In [119]:
# Predicting and converting into Dataframe
y1_predict = pd.DataFrame(grid_model_XGB.predict(X1_test))
Proba = grid_model_XGB.predict_proba(X1_test)
y1_predict.shape

(7170, 1)

## Analyse to find the right cutoff for classification. Default is .5

In [120]:
y1_test.shape

(7170, 1)

In [127]:
# Merge all Data into a single Data Frame index to orirignal split row index
ConsolidatedData = np.concatenate([np.array(y1_test),np.array(y1_predict),np.array(Proba)],axis=1)
ConsolidatedData = pd.DataFrame(ConsolidatedData,columns=['y_test','y_predict','Proba_0','Proba_1'],index=X1_test.index)
ConsolidatedData.head()
#ConsolidatedData.to_excel("E:\\python\Jupyter notebook\Mentoring\Churn-No Churn\ClassifyData.xlsx")

,y_test,y_predict,Proba_0,Proba_1
16230,0.0,0.0,0.933817,0.066183
17941,0.0,0.0,0.878015,0.121985
11299,0.0,0.0,0.812195,0.187805
14975,0.0,0.0,0.967947,0.032053
3068,0.0,0.0,0.945193,0.054807


In [128]:
# Perform sanity check for the Dataframe
print(pd.crosstab(ConsolidatedData.y_test,ConsolidatedData.y_predict))
print(classification_report(ConsolidatedData.y_test,ConsolidatedData.y_predict))
ConsolidatedData.count()

y_predict   0.0  1.0
y_test              
0.0        6948  139
1.0          30   53
              precision    recall  f1-score   support

         0.0       1.00      0.98      0.99      7087
         1.0       0.28      0.64      0.39        83

   micro avg       0.98      0.98      0.98      7170
   macro avg       0.64      0.81      0.69      7170
weighted avg       0.99      0.98      0.98      7170



y_test       7170
y_predict    7170
Proba_0      7170
Proba_1      7170
dtype: int64

In [184]:
# Define Result Lists
Accuracy=[]
Recall=[]
Threshold=[]

ConsolidatedData['y_Pred_new'] = 0
i=20

ConsolidatedData.loc[ConsolidatedData.Proba_1 > i/100 , 'y_Pred_new'] = 1
z=pd.crosstab(ConsolidatedData.y_test,ConsolidatedData.y_Pred_new)
Threshold.append(i/100)
Accuracy.append(((z[0][0]+z[1][1])/(z[0][0]+z[0][1]+z[1][0]+z[1][1]))*100)
Recall.append((z[1][1]/(z[0][1]+z[1][1]))*100)
ConsolidatedData['y_Pred_new'] = 0

print(z)
print("Recall      :",(z[1][1]/(z[0][1]+z[1][1]))*100)
print("Accuracy    :",((z[0][0]+z[1][1])/(z[0][0]+z[0][1]+z[1][0]+z[1][1]))*100)

y_Pred_new     0    1
y_test               
0.0         6181  906
1.0           13   70
Recall      : 84.33734939759037
Accuracy    : 87.18270571827057


In [176]:
'''
# For some stranage Reason this working code is giving error now
# Define Result Lists
Accuracy=[]
Recall=[]
Threshold=[]

# Set the new Column to Zero
ConsolidatedData['y_Pred_new'] = 0

# Loop to evaluate the thersohold
for i in range(1,101,1):
    ConsolidatedData.loc[ConsolidatedData.Proba_1 > i/100 , 'y_Pred_new'] = 1
    z=pd.crosstab(ConsolidatedData.y_test,ConsolidatedData.y_Pred_new)
    Threshold.append(i/100)
    Accuracy.append(((z[0][0]+z[1][1])/(z[0][0]+z[0][1]+z[1][0]+z[1][1]))*100)
    Recall.append((z[1][1]/(z[0][1]+z[1][1]))*100)
    ConsolidatedData['y_Pred_new'] = 0

# Convert the lists into a single Data frame
Result = pd.DataFrame({'Threshold':Threshold,'Accuracy':Accuracy,'Recall':Recall})
'''


KeyError: 1

In [124]:
Result.plot(x='Threshold',
            y=['Accuracy','Recall'],
            kind='line',
            grid=True,
            title="Threshold vs Accuracy, Recall",
            legend = True,
            figsize = (10,5)
            )

NameError: name 'Result' is not defined

In [175]:
# Comment
#Result